<a href="https://colab.research.google.com/github/MacraeSmith/helloAI/blob/main/Assignment_7_metaltest1_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import time
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Define linear model we will use below
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# modified from https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
# get our competing devices ready ... go ahead and init all three here, but ONLY USE ONE during each test
# scroll down below and replace the references to gpu_, tpu_, cpu with whichever device you are testing
# make sure you replace ALL of them
gpu_device = torch.device("cuda")
tpu_device = torch.device("xla")
cpu_device = torch.device("cpu")

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# first hyperparam
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Show some sample data
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


# sample code for working with m1
  # Create a Tensor directly on the mps device
  #x = torch.ones(5, device=mps_device)
  # Or
  #x = torch.ones(5, device="mps")
  # Any operation happens on the GPU
  #y = x * 2

  # Move your model to mps just like any other device


100%|██████████| 26421880/26421880 [00:02<00:00, 12035112.39it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 201491.03it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3760375.82it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 17287651.71it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [2]:
#CPU
model = NeuralNetwork().to(cpu_device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print("starting timer for training using CPU...")
start = time.time()
epochs = 30 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, cpu_device)
print(f"completed training in ... {time.time()-start}s")

print("starting timer for testing using CPU...")
start = time.time()
test(test_dataloader, model, loss_fn, cpu_device)
print(f"completed testing in ... {time.time()-start}s")

starting timer for training using CPU...
Epoch 1
-------------------------------
loss: 2.305162  [   64/60000]
loss: 2.294784  [ 6464/60000]
loss: 2.271259  [12864/60000]
loss: 2.264897  [19264/60000]
loss: 2.247864  [25664/60000]
loss: 2.219598  [32064/60000]
loss: 2.223181  [38464/60000]
loss: 2.192077  [44864/60000]
loss: 2.193119  [51264/60000]
loss: 2.161331  [57664/60000]
Epoch 2
-------------------------------
loss: 2.169448  [   64/60000]
loss: 2.161677  [ 6464/60000]
loss: 2.101401  [12864/60000]
loss: 2.113982  [19264/60000]
loss: 2.063841  [25664/60000]
loss: 2.007838  [32064/60000]
loss: 2.029166  [38464/60000]
loss: 1.957880  [44864/60000]
loss: 1.969398  [51264/60000]
loss: 1.892366  [57664/60000]
Epoch 3
-------------------------------
loss: 1.933451  [   64/60000]
loss: 1.902829  [ 6464/60000]
loss: 1.784965  [12864/60000]
loss: 1.817998  [19264/60000]
loss: 1.713260  [25664/60000]
loss: 1.668354  [32064/60000]
loss: 1.683011  [38464/60000]
loss: 1.592247  [44864/60000]

In [3]:
#GPU
model = NeuralNetwork().to(gpu_device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print("starting timer for training using gPU...")
start = time.time()
epochs = 30 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, gpu_device)
print(f"completed training in ... {time.time()-start}s")

print("starting timer for testing using gPU...")
start = time.time()
test(test_dataloader, model, loss_fn, gpu_device)
print(f"completed testing in ... {time.time()-start}s")

RuntimeError: ignored

In [ ]:
#TPU
model = NeuralNetwork().to(tpu_device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print("starting timer for training using TPU...")
start = time.time()
epochs = 30 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, tpu_device)
print(f"completed training in ... {time.time()-start}s")

print("starting timer for testing using TPU...")
start = time.time()
test(test_dataloader, model, loss_fn, tpu_device)
print(f"completed testing in ... {time.time()-start}s")